# 基于角色的权限控制 RBAC入门

如果使用ｄｂ内置支持需注意`bigchaindb RBAC模块在bigchaindb中没有开箱即用，需要使用kyber-master分支，加载smart asset插件`

另一种方法是在ｓｅｒｖｅｒ端设置验证，感觉更灵活一些。

# 一切都是资产
角色，权限，资源，用户都可以视为资产

## ａｐｐ结构和资产等级
通过资产层次结构用于为用户提供不同资源

## ｌｉｎｋ
定义资产间的父子关系

１.创建父资产，`父ｉｄ为ＣＲＥＡＴＥ事务ｉｄ`

２.父资产为层次结构中顶层资产，父资产ｉｄ作为`ａｐｐ　ｉｄ`

３.在子资产中定义新的保留属性　`ｌｉｎｋ：父资产ｉｄ`，从而定义了资产的父子关系。

例：

## App,Types,Instance实例
ａｐｐ由ｄｂ中的ａｓｓｅｔ表示,位于顶层层级,ａｐｐ有types作为子,ｔｙｐｅｓ有ｉｎｓｔａｎｃｅ.
他们实质都是ｄｂ的 asset.types　ｌｉｎｋ　ｔｏ　ａｐｐ,ｉｎｓｔａｎｃｅｓ　ｌｉｎｋ to 各自的ｔｙｐｅｓ.

ｔｙｐｅｓ 就像ＯＯ　ｓｙｓｔｅｍ（面向对象系统）中的类.他也是唯一ｌｉｎｋ到ａｐｐ的ａｓｓｅｔ.
典型实例：ｒｏｌｅ，users，message,（以及在场景中可以有多个实例的任何事物，车辆，报告等）.

ｉｎｓｔａｎｃｅ是待表一种单个实体类型的对象.例如:与ａｐｐ交互的用户都有为他们创建的ａｓｓｅｔ,
这种ａｓｓｅｔ　ｌｉｎｋｓ to 用户的　ｔｙｐｅ　ａｓｓｅｔ.本质上,每个物理用户都有一个与之关联的用户类型实例.
例子,一个物联网设备会有一个ａｓｓｅｔ代表它,在系统中，这个ａｓｓｅｔ会被连接到各自的父类型,如设备,事物等.

ｎａｍｅｓｐａｃｅ（ｎｓ）是另一个添加到ａｓｓｅｔ.data的属性，用于方便查找特定类型的资产,在ｄｂ中.
ａｐｐ　ａｓｓｅｔ定义ｒｏｏｔ接口,附加ｔｙｐｅｓ到ａｐｐ.非常相似的.ｎｅｔ　ｎｓ　在文件名添加子命名空间.
在这个ＲＢＡＣ设置中,使用ｎｓ属性作为ｔｅｘｔ参数查找特定类型的资产.

如下层次结构：

## 1. App

## 2. Types

## 3. Instances

```
Note:
app{ns,id:create id} 
types{ns,link:app.id,id} 
instances(ns,link:type.id,id)
```

# Permissions 权限

## 现在已经定义了层次结构，问题变为谁有权限link asset？现在需要在asset中定义两个类型的权限。

1.向系统中添加新资源的权限－如新用户，新设备，new instance of a type.

2.对资源执行动作的权限

## To implement permissions

1.添加属性 can_link 到　transaction　的　metdata 中。值是公钥列表或代表用户types的asset id（代表admin group）.

2.在ｄｂ中添加一些服务端验证，以读取can_link属性，验证创建link链接资产的用户公钥是否在允许列表中，或在其钱包中是否只有资产链接到list中的资产ｉｄ。验证通过则记录交易。

样例：

## 1. As a list of public keys

## 2. As the asset id of a user type

## And, here’s the pseudo code for validation on server side:

现在只对有　link　属性的ａｓｓｅｔ强制执行验证。如果这个资产没有与其他资产链接，不会强制执行次验证，并且无需进一步检查即可创建资产。

作者已将故意将　can_link　保存在 metada 中,所以可以使用　TRANSFER 交易添加这个　key　的值，而且可以读最新的值，用于在未来更新这个权限。

第二个设置权限的方式是使用ｄｂ的内置支持crypto-conditions强制执行。
不知道这句翻译的对不对，放原文：
The second set of permissions to act on an asset (TRANSFER) can be enforced using BigchainDB’s inbuilt support for crypto-conditions and is self-explanatory.
不过这个内置的不在主分支，不能开箱即用。

# Putting it all together
 do RBAC on BigchainDB assets　
 使用服务端的权限验证，就不用内置的支持了。
 For example: 
 [Scenario — Vehicle life-cycle management](http://localhost:8888/notebooks/my_python/notebook_demo/Scenario%E2%80%8A%E2%80%94%E2%80%8AVehicle%20life-cycle%20management.ipynb)

２０１８／１２／１３